In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/huggingfacedebertav3variants/khalidalt-DeBERTa-v3-large/spm.model
/kaggle/input/huggingfacedebertav3variants/khalidalt-DeBERTa-v3-large/config.json
/kaggle/input/huggingfacedebertav3variants/khalidalt-DeBERTa-v3-large/README (1).md
/kaggle/input/huggingfacedebertav3variants/khalidalt-DeBERTa-v3-large/README.md
/kaggle/input/huggingfacedebertav3variants/khalidalt-DeBERTa-v3-large/tokenizer_config.json
/kaggle/input/huggingfacedebertav3variants/khalidalt-DeBERTa-v3-large/tokenizer_config (1).json
/kaggle/input/huggingfacedebertav3variants/khalidalt-DeBERTa-v3-large/pytorch_model.bin
/kaggle/input/huggingfacedebertav3variants/khalidalt-DeBERTa-v3-large/special_tokens_map.json
/kaggle/input/huggingfacedebertav3variants/khalidalt-DeBERTa-v3-large/gitattributes.txt
/kaggle/input/huggingfacedebertav3variants/khalidalt-DeBERTa-v3-large/added_tokens.json
/kaggle/input/huggingfacedebertav3variants/deberta-v3-base-squad2/spm.model
/kaggle/input/huggingfacedebertav3variants/deberta-v

In [2]:
import transformers
import datasets
import pandas as pd
import numpy as np

In [3]:
model_checkpoint = "/kaggle/input/huggingfacedebertav3variants/deberta-v3-xsmall"

In [4]:
from datasets import Dataset

In [5]:
df = pd.read_csv('/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv')

In [6]:
train = df[df.prompt_name != 'Car-free cities'].reset_index(drop=True)
valid = df[df.prompt_name == 'Car-free cities'].reset_index(drop=True)
train.head()

,text,label,prompt_name,source,RDizzl3_seven
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False
3,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False
4,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False


In [7]:
not_persuade_df = train[train['source'] != 'persuade_corpus']
persuade_df = train[train['source'] == 'persuade_corpus']
sampled_persuade_df = persuade_df.sample(n=6000, random_state=42)

In [8]:
all_human = set(list(''.join(sampled_persuade_df.text.to_list())))
other = set(list(''.join(not_persuade_df.text.to_list())))

In [9]:
chars_to_remove = ''.join([x for x in other if x not in all_human])
print(chars_to_remove)

🧐📰🙅🛋🚔­🙌必😮🏢😈📷💭🏏🌟👌🥘路🌲该🎅🍕📖😌🧽🚗🛠ā👍🌞🥁使🍎🥪上ち应🧹🐳🦎♂🏟😲🔭🥶🤪🍝🐶☹🎶😍🐧🏜💃🧠🥕“😂🚑🙄🕰。🤷÷😁ã^🌴📚💦😝😬司👧🇪🎩部…😋🐰🌧🤜🌻□🤢’😅ま🐕😭🙏🐒时🐢🛣🐟👦🧦🥯🔬🐬é🕹🚣🇺所╯💔👀手👨影💨😓🤞?🏨🤛🌳💖🏫🤘す🙊🇵🔑🎓🎣🍋在🇧🌷あう🐻🥖ó🏻🍄📧😘🧖😷🥦íは🎸🚂🥔🍽🛀😖🎭😢🤤🔮📱🧭🎵🌠驾🐸🕒🤦せ🏞🎾🏙🏥驶🏼🚚🧡🍣¬に🚨将须🌫🏄🇫🏠🧑🎧🏈👬🤓💡🌽😠😜📄🎄🏀”🌃​🎨合😴注🥗🎬🍗🗳🗣一🔍👥☀🔧唯️💇🙈🇯👫🧙📦🔜😄😆🚪⏰📺⚽🌮📊全🥤🏽💅🤔保🌏💪止💫ü👏🇷💀🎹💤🍓🥑🥩😱意🍷🤖🕵🐭💁🌐😵择🧩集🏃🏔🍜🌌有🌯😃🌱😉🛸🏡🏛取💆📅👇🎉安–🍁👻🇸🏦🎤😊💯🥭🍭道🛑🐴🌄が🕺🐦👂💻🌊💧📹🏆😨🐠💬🍲ん📸是👋用🐱🎢💰🎈和🙀🤫禁👪📞🥜🤩🧚🍰🎠


In [10]:
translation_table = str.maketrans('', '', chars_to_remove)
def remove_chars(s):
    return s.translate(translation_table)
not_persuade_df['text'] = not_persuade_df['text'].apply(remove_chars)

/tmp/ipykernel_47/2013655814.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_persuade_df['text'] = not_persuade_df['text'].apply(remove_chars)


In [11]:
train = pd.concat([not_persuade_df, sampled_persuade_df]).sample(frac=1, random_state=42).reset_index(drop=True)
train.source.value_counts()

source
persuade_corpus                       6000
llama2_chat                           2411
chat_gpt_moth                         2409
mistral7binstruct_v1                  2408
mistral7binstruct_v2                  2406
llama_70b_v1                           984
darragh_claude_v6                      952
darragh_claude_v7                      951
falcon_180b_v1                         899
kingki19_palm                          672
train_essays                           670
cohere-command                         301
palm-text-bison1                       300
radek_500                              250
mistralai/Mistral-7B-Instruct-v0.1     201
NousResearch/Llama-2-7b-chat-hf        200
radekgpt4                              100
Name: count, dtype: int64

In [12]:
ds_train = Dataset.from_pandas(train)
ds_valid = Dataset.from_pandas(valid)

In [13]:
ds_train

Dataset({
    features: ['text', 'label', 'prompt_name', 'source', 'RDizzl3_seven'],
    num_rows: 22114
})

In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [15]:
def preprocess_function(examples):
    return tokenizer(examples['text'], max_length=128, padding=True, truncation=True)

In [16]:
ds_train_enc = ds_train.map(preprocess_function, batched=True)

  0%|          | 0/23 [00:00<?, ?ba/s]

In [17]:
ds_valid_enc = ds_valid.map(preprocess_function , batched = True)

  0%|          | 0/5 [00:00<?, ?ba/s]

In [18]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels = num_labels)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/huggingfacedebertav3variants/deberta-v3-xsmall and are newly initialized: ['pooler.dense.weight', 'classifier.weight', 'classifier.bias', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
metric_name  ="roc_auc"
model_name = "deberta-xsmall"
train_batch_size = 4
eval_batch_size = 32
grad_acc =4

In [20]:
num_steps = len(train)//(train_batch_size * grad_acc)
num_steps

1382

In [21]:
args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "steps",
    save_strategy= "steps",
    eval_steps = num_steps //3,
    save_steps = num_steps //3,
    learning_rate=2e-5,
    per_device_train_batch_size = train_batch_size,
    per_device_eval_batch_size = eval_batch_size,
    gradient_accumulation_steps = grad_acc,
    num_train_epochs =1,
    weight_decay = 0.01,
    load_best_model_at_end = False,
    metric_for_best_model = metric_name,
    report_to ='none',)

In [22]:
from sklearn.metrics import roc_auc_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = np.exp(logits) / np.sum(np.exp(logits), axis = -1, keepdims = True)
    auc = roc_auc_score(labels, probs[:,1], multi_class = 'ovr')
    return {"roc_auc": auc}
    

In [23]:
trainer = Trainer(
    model,
    args,
    train_dataset = ds_train_enc,
    eval_dataset = ds_valid_enc,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics)

In [24]:
trainer.train()

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Roc Auc
460,No log,0.329228,0.993091
920,0.151300,0.179845,0.999158
1380,0.049900,0.255289,0.997103


TrainOutput(global_step=1382, training_loss=0.081633302823852, metrics={'train_runtime': 8767.1164, 'train_samples_per_second': 2.522, 'train_steps_per_second': 0.158, 'total_flos': 364157494247424.0, 'train_loss': 0.081633302823852, 'epoch': 1.0})